In [5]:
from pyopenms import *
import os
import glob

In [ ]:
path= "results/interim"
if not os.path.exists(path):
    os.mkdir(path)

In [24]:
# 1) Precursor Mass Correction

input_original_files = glob.glob('Example_data/*.mzML')

for filename in input_original_files:
    print('Processing: ', filename)
    exp = MSExperiment()
    MzMLFile().load(filename, exp)
    exp.sortSpectra(True)
    delta_mzs= []
    mzs = []
    rts= []
    PrecursorCorrection.correctToHighestIntensityMS1Peak(exp, 100.0, True, delta_mzs, mzs, rts)
    MzMLFile().store(os.path.join(path, 'PCpeak_' + os.path.basename(filename)), exp)
print('Finished Precursor Mass Correction')

Processing:  Example_data/Actino_1.mzML
Processing:  Example_data/Actino_2.mzML
Finished Precursor Mass Correction


In [25]:
# 2) Feature Detection

input_mzml_files = glob.glob('results/interim/PCpeak_*.mzML')

# 2.1) Mass trace detection

for filename in input_mzml_files:
    print("Mass Trace Detection: ", filename)
    exp = MSExperiment()
    MzMLFile().load(filename, exp)
    exp.sortSpectra(True)
    mass_traces = []
    mtd = MassTraceDetection()
    mtd_par = mtd.getDefaults()
    mtd_par.setValue("mass_error_ppm", 10.0) 
    mtd_par.setValue("noise_threshold_int", 1.0e04)
    mtd.setParameters(mtd_par)
    mtd.run(exp, mass_traces, 0)

# 2) Elution peak detection
    print("Elution Peak Detection: ", filename)
    mass_traces_split = []
    mass_traces_final = []
    epd = ElutionPeakDetection()
    epd_par = epd.getDefaults()
    epd_par.setValue("width_filtering", "fixed")
    epd.setParameters(epd_par)
    epd.detectPeaks(mass_traces, mass_traces_split)
     
    if (epd.getParameters().getValue("width_filtering") == "auto"):
          epd.filterByPeakWidth(mass_traces_split, mass_traces_final)
    else:
          mass_traces_final = mass_traces_split

# 3) Feature detection
    print("Feature Detection: ", filename)
    feature_map_FFM = FeatureMap()
    feat_chrom = []
    ffm = FeatureFindingMetabo()
    ffm_par = ffm.getDefaults() 
    ffm_par.setValue("isotope_filtering_model", "none")
    ffm_par.setValue("remove_single_traces", "true")
    ffm_par.setValue("mz_scoring_by_elements", "false")
    ffm_par.setValue("report_convex_hulls", "true")
    ffm.setParameters(ffm_par)
    ffm.run(mass_traces_final, feature_map_FFM, feat_chrom)
    feature_map_FFM.setUniqueIds()
    feature_map_FFM.setPrimaryMSRunPath([filename.encode()])
    print(filename[7:-5] + ".featureXML")
    FeatureXMLFile().store(os.path.join(path, os.path.basename(filename)[7:-5] + ".featureXML"), feature_map_FFM)
    
print("Finished Feature Detection")

Mass Trace Detection:  results/interim/PCpeak_Actino_1.mzML
Progress of 'mass trace detection':
-- done [took 0.16 s (CPU), 0.17 s (Wall)] -- 
Elution Peak Detection:  results/interim/PCpeak_Actino_1.mzML
Progress of 'elution peak detection':
Feature Detection:  results/interim/PCpeak_Actino_1.mzML
-- done [took 1.11 s (CPU), 1.11 s (Wall)] -- 
Progress of 'assembling mass traces to features':
/interim/PCpeak_Actino_1.featureXML
Mass Trace Detection:  results/interim/PCpeak_Actino_2.mzML
-- done [took 0.80 s (CPU), 0.77 s (Wall)] -- 
Progress of 'mass trace detection':
Elution Peak Detection:  results/interim/PCpeak_Actino_2.mzML
-- done [took 0.09 s (CPU), 0.11 s (Wall)] -- 
Progress of 'elution peak detection':
-- done [took 0.61 s (CPU), 0.56 s (Wall)] -- 
Feature Detection:  results/interim/PCpeak_Actino_2.mzML
Progress of 'assembling mass traces to features':/interim/PCpeak_Actino_2.featureXML

-- done [took 0.35 s (CPU), 0.33 s (Wall)] -- 
Finished Feature Detection


In [ ]:
input_features = sorted(glob.glob('results/interim/*.featureXML'))

for consensus in input_features:
    fmap = FeatureMap()
    FeatureXMLFile().load(consensus, fmap)
    DF= fmap.get_df()
    DF= DF.drop(columns=["ID_native_id", "peptide_score", "peptide_sequence", "ID_filename"])
    DF=DF.rename(columns={"f1":"intensity"})
print("example:", os.path.basename(consensus))
display(DF)